# Interim Operational Capability 1

## Set up.

### Import packages.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [110]:
designs = ty.Designs("data")

In [111]:
investments = ty.Investments("data")

### Compile the production and metric functions for each technology in the dataset.

In [112]:
designs.compile()

## Examine the data.

### The `functions` table specifies where the Python code for each technology resides.

In [113]:
designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
PV Generic,numpy,pv_residential_generic,capital_cost,fixed_cost,production,metrics,generic residential PV


### The `indices` table defines the subscripts for variables.

In [114]:
designs.indices.drop("Offset", axis = 1)

Description Notes
Technology Type    Index                                      
PV Generic Capital BoS                 balance of system      
                   Inverter             system inverters      
                   Module                  system module      
           Fixed   System                   whole system      
           Input   Strategic         strategic materials      
           Metric  Capital                  capital cost      
                   Efficiency          module efficiency      
                   GHG              greenouse gas offset      
                   Hazardous    hazardous waste produced      
                   LCOE         levelized cost of energy      
                   Lifetime              module lifetime      
                   Strategic        strategic metal used      
                   Yield                  specific yield      
           Output  Electricity     electricity generated      
                   GHG             greenhouse gas offset      
                   Hazardous    hazardous waste produced

### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [115]:
designs.designs

Value  \
Technology Scenario     Variable          Index                                               
PV Generic CIGS 0       Input             Strategic     st.triang(0.5, loc=2520, scale=500)   
                        Input efficiency  Strategic    st.triang(0.5, loc=0.70, scale=0.05)   
                        Input price       Stategic                                        0   
                        Lifetime          BoS                                             1   
                                          Inverter                                        1   
...                                                                                     ...   
           Soft Costs 2 Output efficiency Hazardous                                       1   
                        Output price      Electricity                                     0   
                                          GHG                                             0   
                                          Hazardous                                       0   
                        Scale             NaN                                             1   

                                                                 Units  \
Technology Scenario     Variable          Index                          
PV Generic CIGS 0       Input             Strategic           g/system   
                        Input efficiency  Strategic              %/100   
                        Input price       Stategic                 $/g   
                        Lifetime          BoS          system-lifetime   
                                          Inverter     system-lifetime   
...                                                                ...   
           Soft Costs 2 Output efficiency Hazardous              %/100   
                        Output price      Electricity            $/kWh   
                                          GHG                  $/gCO2e   
                                          Hazardous                $/g   
                        Scale             NaN            system/system   

                                                                                                 Notes  
Technology Scenario     Variable          Index                                                         
PV Generic CIGS 0       Input             Strategic                                    zero by default  
                        Input efficiency  Strategic                                         no scaling  
                        Input price       Stategic                                     zero by default  
                        Lifetime          BoS                                per-lifetime computations  
                                          Inverter                           per-lifetime computations  
...                                                                                                ...  
           Soft Costs 2 Output efficiency Hazardous    see parameter table for individual efficiencies  
                        Output price      Electricity                   not tracking electricity price  
                                          GHG                             not tracking hazardous waste  
                                          Hazardous                       not tracking hazardous waste  
                        Scale             NaN                                               no scaling  

[299 rows x 3 columns]

### The `parameters` table contains additional techno-economic parameters for each technology.

In [116]:
designs.parameters.drop("Offset", axis = 1)

Value  \
Technology Scenario     Parameter                                                    
PV Generic CIGS 0       Benchmark LCOC                                     2.51977   
                        Benchmark LCOE                                    0.086773   
                        Customer Acquisition   st.triang(0.5, loc=2000, scale=0.2)   
                        Direct Labor           st.triang(0.5, loc=2000, scale=0.2)   
                        Discount Rate                                         0.07   
...                                                                            ...   
           Soft Costs 2 Module Lifetime       st.triang(0.5, loc=25, scale=0.0025)   
                        Module O&M Fixed       st.triang(0.5, loc=20, scale=0.002)   
                        Module Soiling Loss   st.triang(0.5, loc=0.05, scale=5e-6)   
                        Permitting              st.triang(0.5, loc=400, scale=100)   
                        System Size                                             36   

                                                 Units Notes  
Technology Scenario     Parameter                             
PV Generic CIGS 0       Benchmark LCOC           $/Wdc        
                        Benchmark LCOE           $/kWh        
                        Customer Acquisition  $/system   BCA  
                        Direct Labor          $/system   BLR  
                        Discount Rate           1/year    DR  
...                                                ...   ...  
           Soft Costs 2 Module Lifetime             yr   MLT  
                        Module O&M Fixed       $/kW/yr   MOM  
                        Module Soiling Loss      %/100   MSL  
                        Permitting            $/system   BPR  
                        System Size                m^2   SSZ  

[552 rows x 3 columns]

### The `results` table specifies the units of measure for results of computations.

In [117]:
designs.results

Units Notes
Technology Variable Index                           
PV Generic Cost     Cost              $/system      
           Metric   Capital             Δ$/Wdc      
                    Efficiency           %/100      
                    GHG          ΔgCO2e/system      
                    Hazardous            g/kWh      
                    LCOE                Δ$/kWh      
                    Lifetime                yr      
                    Strategic            g/kWh      
                    Yield                hr/yr      
           Output   Electricity     kWh/system      
                    GHG          ΔgCO2e/system      
                    Hazardous         g/system

### The `tranches` table specifies multually exclusive possibilities for investments: only one `Tranch` may be selected for each `Category`.

In [118]:
investments.tranches

Amount Notes
Category          Tranche             Scenario                            
CIGS              CIGS 0              CIGS 0                     0.0      
                  CIGS 1              CIGS 1               1500000.0      
                  CIGS 2              CIGS 2               3000000.0      
CdTe              CdTe 0              CdTe 0                     0.0      
                  CdTe 1              CdTe 1               3000000.0      
                  CdTe 2              CdTe 2               6000000.0      
GaAs              GaAs 0              GaAs 0                     0.0      
                  GaAs 1              GaAs 1               5000000.0      
                  GaAs 2              GaAs 2               7500000.0      
InGaP             InGaP 0             InGaP 0                    0.0      
                  InGaP 1             InGaP 1              3500000.0      
                  InGaP 2             InGaP 2              9500000.0      
Perovskite        Perovskite 0        Perovskite 0               0.0      
                  Perovskite 1        Perovskite 1         7500000.0      
                  Perovskite 2        Perovskite 2         9500000.0      
Polysilicon       Polysilicon 0       Polysilicon 0              0.0      
                  Polysilicon 1       Polysilicon 1        2500000.0      
                  Polysilicon 2       Polysilicon 2        4500000.0      
Power Electronics Power Electronics 0 Polysilicon 0              0.0      
                  Power Electronics 1 Power Electronics 1  1000000.0      
                  Power Electronics 2 Power Electronics 2  5000000.0      
Soft Costs        Soft Costs 0        Polysilicon 0              0.0      
                  Soft Costs 1        Soft Costs 1         1000000.0      
                  Soft Costs 2        Soft Costs 2         5000000.0

### The `investments` table bundles a consistent set of tranches (one per category) into an overall investment.

In [119]:
investments.investments

,,,Notes
Investment,Category,Tranche,


## Evaluate the scenarios in the dataset.

In [120]:
scenario_results = designs.evaluate_scenarios(sample_count=500)

In [121]:
scenario_results.xs(
    "PV Generic"
).groupby(
    ["Scenario", "Variable", "Index"]
).aggregate(
    np.mean
).xs(
    "Metric", level = 1
).pivot_table(
    index = "Scenario", columns = "Index"
)

Value                                 \
Index                     Capital Efficiency       GHG Hazardous   
Scenario                                                           
CIGS 0              -7.894838e-01   0.234880 -0.000449  0.055617   
CIGS 1              -2.053988e-01   0.265220 -0.000449  0.044241   
CIGS 2               1.162300e-01   0.274775 -0.000449  0.037959   
CdTe 0              -4.350222e-01   0.234450 -0.000449  0.054182   
CdTe 1              -1.044040e-01   0.254844 -0.000449  0.045925   
CdTe 2               2.413545e-01   0.284414 -0.000449  0.037006   
GaAs 0              -2.428062e-01   0.312296 -0.000449  0.075718   
GaAs 1              -1.363030e-02   0.314397 -0.000449  0.073980   
GaAs 2               4.640476e-01   0.344852 -0.000449  0.066145   
InGaP 0              3.833736e-01   0.403722 -0.000449  0.023522   
InGaP 1              7.375983e-01   0.410066 -0.000449  0.018427   
InGaP 2              9.518421e-01   0.397516 -0.000449  0.018078   
Perovskite 0        -1.134395e+00   0.181374 -0.000449  0.024834   
Perovskite 1        -3.849125e-01   0.224214 -0.000449  0.017019   
Perovskite 2         3.451298e-01   0.294851 -0.000449  0.008638   
Polysilicon 0        2.140904e-01   0.232542 -0.000449  0.247307   
Polysilicon 1        3.315448e-01   0.244430 -0.000449  0.205394   
Polysilicon 2        5.745080e-01   0.275120 -0.000449  0.149557   
Power Electronics 1  2.891479e-01   0.232824 -0.000449  0.235312   
Power Electronics 2  3.649610e-01   0.232308 -0.000449  0.228535   
Reference           -4.304827e-07   0.208000 -0.000449  0.275327   
Soft Costs 1         3.063810e-01   0.233618 -0.000449  0.246223   
Soft Costs 2         4.711288e-01   0.233380 -0.000449  0.246483   

                                                                     
Index                        LCOE   Lifetime Strategic        Yield  
Scenario                                                             
CIGS 0              -6.306403e-02  25.001249  0.013825   963.334243  
CIGS 1              -3.785841e-02  25.001241  0.011637   969.499550  
CIGS 2              -2.174859e-02  25.001265  0.010562   992.034518  
CdTe 0              -1.359586e-02  25.001236  0.031685  1278.373701  
CdTe 1              -5.415598e-04  25.001215  0.025668  1318.049577  
CdTe 2               1.182150e-02  25.001294  0.021721  1350.939368  
GaAs 0              -7.581908e-03  25.001226  0.007312  1278.371851  
GaAs 1              -1.441918e-04  25.001266  0.006191  1282.303410  
GaAs 2               1.518507e-02  25.001229  0.004878  1289.993391  
InGaP 0             -1.481606e-03  20.498535  0.003091  1310.560055  
InGaP 1              1.242066e-02  20.486901  0.002358  1323.880788  
InGaP 2              2.037385e-02  20.506564  0.002380  1323.738071  
Perovskite 0        -7.886534e-02  17.555317  0.007610  1335.383717  
Perovskite 1        -4.545850e-02  17.419037  0.005280  1359.771907  
Perovskite 2        -1.450781e-02  17.416189  0.003619  1359.763595  
Polysilicon 0        6.713862e-03  25.001272  0.000000  1278.374264  
Polysilicon 1        1.153258e-02  25.001226  0.000000  1297.869707  
Polysilicon 2        2.109504e-02  25.001204  0.000000  1338.853594  
Power Electronics 1  1.275730e-02  25.001269  0.000000  1342.237812  
Power Electronics 2  1.724145e-02  25.001248  0.000000  1385.192305  
Reference           -1.946058e-07  25.000000  0.000000  1278.202148  
Soft Costs 1         9.601346e-03  25.001290  0.000000  1278.371130  
Soft Costs 2         1.475623e-02  25.001237  0.000000  1278.377702

### Save results.

In [ ]:
scenario_results.to_csv("output/example-scenario.csv")

### Plot GHG metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "GHG"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="GHG Reduction [gCO2e / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Plot LCOE metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "LCOE"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="LCOE Reduction [USD / kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Plot labor metric.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    data=scenario_results.xs(
        ["Metric", "Labor"],
        level=["Variable", "Index"]
    ).reset_index()[["Scenario", "Value"]],
    order=[
        "2015 Actual"              ,
        "Module Slow Progress"      ,
        "Module Moderate Progress"  ,
        "Module Fast Progress"      ,
        "Inverter Slow Progress"    ,
        "Inverter Moderate Progress",
        "Inverter Fast Progress"    ,
        "BoS Slow Progress"         ,
        "BoS Moderate Progress"     ,
        "BoS Fast Progress"         ,
    ]
)
g.set(ylabel="Labor Increase [USD / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

## Evaluate the investments in the dataset.

In [ ]:
investment_results = investments.evaluate_investments(designs, sample_count=50)

### Costs of investments.

In [ ]:
investment_results.amounts

### Benefits of investments.

In [ ]:
investment_results.metrics.xs(1, level="Sample", drop_level=False)

In [ ]:
investment_results.summary.xs(1, level="Sample", drop_level=False)

### Save results.

In [ ]:
investment_results.amounts.to_csv("output/residential_pv_multiobjective/example-investment-amounts.csv")

In [ ]:
investment_results.metrics.to_csv("output/residential_pv_multiobjective/example-investment-metrics.csv")

### Plot GHG metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "GHG",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="GHG Reduction [gCO2e / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

### Plot LCOE metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "LCOE",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="LCOE Reduction [USD / kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

### Plot labor metric.

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Labor",
        level="Index"
    ).reset_index()[["Investment", "Value"]],
    order=[
        "Low R&D"   ,
        "Medium R&D",
        "High R&D"  ,
    ]
)
g.set(ylabel="Labor Increase [USD / system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

## Multi-objective decision analysis.

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

Display the cost of each tranche.

In [ ]:
tranche_results.amounts

Display the metrics for each tranche.

In [ ]:
tranche_results.summary

Save the results.

In [ ]:
tranche_results.amounts.to_csv("output/residential_pv_multiobjective/example-tranche-amounts.csv")
tranche_results.summary.to_csv("output/residential_pv_multiobjective/example-tranche-summary.csv")

### Fit a response surface to the results.

The response surface interpolates between the discrete set of cases provided in the expert elicitation. This allows us to study funding levels intermediate between those scenarios.

In [ ]:
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Here are the categories of investment and the maximum amount that could be invested in each:

In [ ]:
evaluator.max_amount

Here are the metrics and their units of measure:

In [ ]:
evaluator.units

#### Example interpolation.

Let's evaluate the case where each category is invested in at half of its maximum amount.

In [ ]:
example_investments = evaluator.max_amount / 2
example_investments

In [ ]:
evaluator.evaluate(example_investments)

Let's evaluate the mean instead of outputing the whole distribution.

In [ ]:
evaluator.evaluate_statistic(example_investments, np.mean)

Here is the standard deviation:

In [ ]:
evaluator.evaluate_statistic(example_investments, np.std)

A risk-averse decision maker might be interested in the 10% percentile:

In [ ]:
evaluator.evaluate_statistic(example_investments, lambda x: np.quantile(x, 0.1))

### ε-Constraint multiobjective optimization

In [ ]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

In order to meaningfully map the decision space, we need to know the maximum values for each of the metrics.

In [ ]:
metric_max = optimizer.max_metrics()
metric_max

#### Example optimization.

Limit spending to $3M.

In [ ]:
investment_max = 3e6

Require that the GHG reduction be at least 40 gCO2e/system and that the Labor wages not decrease.

In [ ]:
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])
metric_min

Compute the ε-constrained maximum for the LCOE.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = np.mean       ,
)
optimum.exit_message

Here are the optimal spending levels:

In [ ]:
np.round(optimum.amounts)

Here are the three metrics at that optimum:

In [ ]:
optimum.metrics

*Thus, by putting all of the investment into Module R&D, we can expected to achieve a mean 3.75 ¢/kWh reduction in LCOE under the GHG and Labor constraints.*

It turns out that there is no solution for these constraints if we evaluate the 10th percentile of the metrics, for a risk-averse decision maker.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = lambda x: np.quantile(x, 0.1),
)
optimum.exit_message

Let's try again, but with a less stringent set of constraints, only constraining GHG somewhat  but not Labor at all.

In [ ]:
optimum = optimizer.maximize(
    "LCOE"                                                         ,
    total_amount = investment_max                                  ,
    min_metric   = pd.Series([30], name = "Value", index = ["GHG"]),
    statistic    = lambda x: np.quantile(x, 0.1)                   ,
)
optimum.exit_message

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics

### Pareto surfaces.

##### Metrics constrained by total investment.

In [ ]:
pareto_amounts = None
for investment_max in np.arange(1e6, 9e6, 0.5e6):
    metrics = optimizer.max_metrics(total_amount = investment_max)
    pareto_amounts = pd.DataFrame(
        [metrics.values]                                         ,
        columns = metrics.index.values                           ,
        index   = pd.Index([investment_max / 1e6], name = "Investment [M$]"),
    ).append(pareto_amounts)
pareto_amounts

In [ ]:
sb.relplot(
    x         = "Investment [M$]",
    y         = "Value"          ,
    col       = "Metric"         ,
    kind      = "line"           ,
    facet_kws = {'sharey': False},
    data      = pareto_amounts.reset_index().melt(id_vars = "Investment [M$]", var_name = "Metric", value_name = "Value")
)

*We see that the LCOE metric saturates more slowly than the GHG and Labor ones.*

##### GHG vs LCOE, constrained by total investment.

In [ ]:
investment_max = 3
pareto_ghg_lcoe = None
for lcoe_min in 0.95 * np.arange(0.5, 0.9, 0.05) * pareto_amounts.loc[investment_max, "LCOE"]:
    optimum = optimizer.maximize(
        "GHG",
        max_amount   = pd.Series([0.9e6, 3.0e6, 1.0e6], name = "Amount", index = ["BoS R&D", "Inverter R&D", "Module R&D"]),
        total_amount = investment_max * 1e6                                 ,
        min_metric   = pd.Series([lcoe_min], name = "Value", index = ["LCOE"]),
    )
    pareto_ghg_lcoe = pd.DataFrame(
        [[investment_max, lcoe_min, optimum.metrics["LCOE"], optimum.metrics["GHG"], optimum.exit_message]],
        columns = ["Investment [M$]", "LCOE (min)", "LCOE", "GHG", "Result"]                               ,
    ).append(pareto_ghg_lcoe)
pareto_ghg_lcoe = pareto_ghg_lcoe.set_index(["Investment [M$]", "LCOE (min)"])
pareto_ghg_lcoe

In [ ]:
sb.relplot(
    x = "LCOE",
    y = "GHG",
    kind = "scatter",
    data = pareto_ghg_lcoe#[pareto_ghg_lcoe.Result == "Optimization terminated successfully."]
)

*The three types of investment are too decoupled to make an interesting pareto frontier, and we also need a better solver if we want to push to lower right.*

## Run the interactive explorer for the decision space.

<font color="red">Make sure the the `tk` package is installed on your machine.</font> Here is the Anaconda link: https://anaconda.org/anaconda/tk.

In [ ]:
w = ty.DecisionWindow(evaluator)
w.mainloop()

A new window should open that looks like the image below. Moving the sliders will cause a recomputation of the boxplots.

In [ ]:
Image("residential_pv_multiobjective_gui.png")